# Encriptacion y Desencriptacion usando ECC

### Herramientas usadas: python 3, ECC como generador de llave publica y privada y AES como metodo de encriptacion y desencriptacion

#### Explicacion:
El codigo lo que hace es generar valores enteros entre 23 y 37, simulando ser datos de temperaturas obtenidas por el sensor, que luego se almacenan en un bloc de notas de nombre "temperaturas.txt" que tambien es creado por el codigo.
Luego se definen las funciones de encriptacion y desencriptacion con AES-GCM

In [34]:
import random
import time
import requests
import io
import git
from tinyec import registry
from Crypto.Cipher import AES
import hashlib, secrets, binascii
import pickle 

In [53]:
with open("/Users/Chris-OSX/Cosas/temperaturas.txt", "w") as w:
    for n in range(0, 3):
        w.write(str(random.randint(23, 37))+ " °C\n")
    

In [36]:
def encrypt_AES_GCM(msg, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM)
  ciphertext, authTag = aesCipher.encrypt_and_digest(msg)
  return (ciphertext, aesCipher.nonce, authTag)


def ecc_point_to_256_bit_key(point):
  sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
  sha.update(int.to_bytes(point.y, 32, 'big'))
  return sha.digest()

curve = registry.get_curve('brainpoolP256r1')

def encrypt_ECC(msg, pubKey):
  ciphertextPrivKey = secrets.randbelow(curve.field.n) # a
  sharedECCKey = ciphertextPrivKey * pubKey #a·B = T
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  ciphertext, nonce, authTag = encrypt_AES_GCM(msg, secretKey)
  ciphertextPubKey = ciphertextPrivKey * curve.g #A
  return (ciphertext, nonce, authTag, ciphertextPubKey)


In [51]:
repo = git.Repo("/Users/Chris-OSX/Cosas/TEL252_Central_de_datos")

In [54]:
f = open("/Users/Chris-OSX/Cosas/temperaturas.txt", "rb")
while(True):
    msg = f.readline()
    print("original msg:", msg.decode('UTF-8'))

    time.sleep(2)
    repo.remotes.origin.pull()

    with open('pub_keyB.pickle', 'rb') as file_pubKeyB:
      pubKeyB = pickle.load(file_pubKeyB) #B

    encryptedMsg = encrypt_ECC(msg, pubKeyB)
    with open('encrypt_data.pickle', 'wb') as file_encrypt_data:
      pickle.dump(encryptedMsg, file_encrypt_data)

    encryptedMsgObj = {
    'ciphertext': binascii.hexlify(encryptedMsg[0]),
    'nonce': binascii.hexlify(encryptedMsg[1]),
    'authTag': binascii.hexlify(encryptedMsg[2]),
    'ciphertextPubKey': hex(encryptedMsg[3].x) + hex(encryptedMsg[3].y % 2)[2:]
    }
    print("encrypted msg:", encryptedMsgObj)
    
    m = f"{count} Data update"
    repo.git.add(repo.working_dir)
    commit_output = repo.git.commit(m=m)
    push_output = repo.git.push('--set-upstream', repo.remote().name, "main")
    
    time.sleep(5)
    if not msg:
        break
f.close()


original msg: 25 °C

encrypted msg: {'ciphertext': b'7d119ff686734c', 'nonce': b'80e1c9a15a4e8394ba2e2c619ffadb20', 'authTag': b'dca01c8bf5141fcb35cc3f149da8e0e5', 'ciphertextPubKey': '0x6224d62c614be0f5684bb9c16864ac767d5ac2a94ea5b67ee95cedcb01cfc8b71'}
original msg: 29 °C

encrypted msg: {'ciphertext': b'1e7d630c394178', 'nonce': b'15069fa59f16efac9cf5c5ff12cbd5f4', 'authTag': b'cd8b14d68e9421006e219d70ccffbae0', 'ciphertextPubKey': '0x380eb1ccbd2d80ce584b9145ebaca4cdcc74470741fa0c0238fb146200427d960'}


GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git push --set-upstream origin main
  stderr: 'To https://github.com/Chrismsz/TEL252_Central_de_datos.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/Chrismsz/TEL252_Central_de_datos.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.'

In [38]:
repo.remotes.origin.pull()

In [39]:
#!git commit -m "Test 1"

In [40]:
#!git config --global user.email "christopher.silva@sansano.usm.cl"

In [41]:
#!git config --global user.name "Chrismsz"

In [42]:
!cd TEL252_Central_de_datos

/bin/bash: line 0: cd: TEL252_Central_de_datos: No such file or directory


In [43]:
repo.git.add(repo.working_dir)
commit_output = repo.git.commit(m="Data Update")
push_output = repo.git.push('--set-upstream', repo.remote().name, "main")

GitCommandError: Cmd('git') failed due to: exit code(1)
  cmdline: git commit -m Data Update
  stdout: 'On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean'

In [ ]:
cd ..

In [ ]:
!git remote set-url origin https://{Chrismsz}:{Esta pass es para git}@github.com/{Chrismsz}/https://github.com/Chrismsz/TEL252_Central_de_datos.git